## 二维卷积层
卷积神经网络$（convolutional$ $ neural$ $ network）$是含有卷积层$（convolutional$ $ layer）$的神经网络。本章中介绍的卷积神经网络均使用最常见的二维卷积层。它有高和宽两个空间维度，常用来处理图像数据。

### 二维互相关运算
虽然卷积层得名于卷积$（convolution）$运算，但我们通常在卷积层中使用更加直观的互相关$（cross-correlation）$运算。在二维卷积层中，一个二维输入数组和一个二维核$（kernel）$数组通过互相关运算输出一个二维数组。举例如下：

<center>
$ \begin{pmatrix} 0&1&2\\3&4&5\\6&7&8 \end{pmatrix} $*$ \begin{pmatrix} 0&1\\2&3 \end{pmatrix} $=$ \begin{pmatrix} 19&25\\37&43 \end{pmatrix} $
</center>

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。

上面的互相关运算如下：
<center>
    $ 0\times 0 + 1\times 1+3\times 2+4\times 3=19 $
</center>
<center>
    $ 1\times 0 + 2\times 1+4\times 2+5\times 3=25 $
</center>
<center>
    $ 3\times 0 + 4\times 1+6\times 2+7\times 3=37 $
</center>
<center>
    $ 4\times 0 + 5\times 1+7\times 2+8\times 3=43 $
</center>

下面我们将上述过程实现在$corr2d$函数里。它接受输入数组$X$与核数组$K$，并输出数组$Y$。

In [1]:
import torch
from torch import nn
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j  in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w]*K).sum()
    return Y

In [2]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
K = torch.tensor([[0,1],[2,3]])
Y = corr2d(X, K)
Y

tensor([[19., 25.],
        [37., 43.]])

### 二维卷积层
二维卷积层将输入和卷积核做互相关运算，并**加上一个标量偏差**来得到输出。卷积层的**模型参数包括了卷积核和标量偏差**。在训练模型的时候，通常我们先对卷积核随机初始化，然后**不断迭代卷积核和偏差**。

下面基于$corr2d$函数来实现一个自定义的二维卷积层。在构造函数$\_\_init\_\_$里我们声明$weight$和$bias$这两个模型参数。前向计算函数$forward$则是直接调用$corr2d$函数再加上偏差。

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, X):
        return corr2d(X, self.weight)+self.bias

卷积窗口形状为$p\times q$的卷积层称为$p\times q$卷积层。同样，$p\times q$卷积或$p \times q$卷积核说明卷积核的高和宽分别为$p$和$q$。

### 图像中物体边缘检测

下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，**即找到像素变化的位置**。首先我们构造一张$6\times 8$的图像（即高和宽分别为$6$像素和$8$像素的图像）。它中间$4$列为黑$（0）$，其余为白$（1）$。

In [4]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

然后我们构造一个高和宽分别为$1$和$2$的卷积核$K$。当它与输入做互相关运算时，如果横向相邻元素相同，输出为$0$；否则输出为非$0$。

In [5]:
K = torch.tensor([[1,-1]], dtype=torch.float32)

下面将输入$X$和我们设计的卷积核$K$做互相关运算。可以看出，我们将从白到黑的边缘和从黑到白的边缘分别检测成了$1$和$-1$。其余部分的输出全是$0$。

In [6]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

由此，我们可以看出，卷积层可通过**重复使用卷积核**有效地**表征局部空间**。

### 通过数据学习核数组
最后我们来看一个例子，它使用物体边缘检测中的输入数据$X$和输出数据$Y$来学习我们构造的核数组$K$。我们首先构造一个卷积层，其卷积核将被**初始化成随机数组**。接下来在每一次迭代中，我们使用平方误差来比较$Y$和卷积层的输出，然后计算梯度来更新权重。

In [7]:
conv2d = Conv2D(kernel_size=(1,2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y-Y_hat)**2).sum()
    l.backward()
    
    conv2d.weight.data -= lr*conv2d.weight.grad
    conv2d.bias.data -= lr*conv2d.bias.grad
    
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i+1)%5 == 0:
        print('step %d, loss %.3f'%(i+1,l.item()))

step 5, loss 4.104
step 10, loss 0.745
step 15, loss 0.163
step 20, loss 0.040


可以看到，$20$次迭代后误差已经降到了一个比较小的值。现在来看一下学习到的卷积核的参数。

可以看到，学到的卷积核的权重参数与我们之前定义的核数组$K$较接近，而偏置参数接近$0$。

In [8]:
print("weight: ", conv2d.weight.data)
print("bias: ", conv2d.bias.data)

weight:  tensor([[ 1.0571, -1.0408]])
bias:  tensor([-0.0091])


### 互相关运算和卷积运算
实际上，卷积运算与互相关运算类似。**为了得到卷积运算的输出，我们只需将核数组左右翻转并上下翻转，再与输入数组做互相关运算**。可见，卷积运算和互相关运算虽然类似，但如果它们使用相同的核数组，对于同一个输入，输出往往并不相同。

那么，你也许会好奇卷积层为何能使用互相关运算替代卷积运算。其实，在深度学习中核数组都是学出来的：**卷积层无论使用互相关运算或卷积运算都不影响模型预测时的输出**。为了解释这一点，假设卷积层使用互相关运算学出图5.1中的核数组。设其他条件不变，使用卷积运算学出的核数组即图5.1中的核数组按上下、左右翻转。也就是说，图5.1中的输入与学出的已翻转的核数组再做卷积运算时，依然得到图5.1中的输出。为了与大多数深度学习文献一致，如无特别说明，本书中提到的卷积运算均指互相关运算。

**感觉深度学习中的卷积运算实际上是互相关运算是个面试题考点。**

### 特征图和感受野
二维卷积层输出的二维数组可以看作是输入在空间维度（宽和高）上某一级的表征，也叫特征图$（feature map）$。影响元素$x$的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做$x$的感受野$（receptive field）$。

图5.1为例，输入中阴影部分的四个元素是输出中阴影部分元素的感受野。我们将图5.1中形状为$2\times 2$的输出记为$Y$，并考虑一个更深的卷积神经网络：将$Y$与另一个形状为$2\times 2$的核数组做互相关运算，输出单个元素$z$。那么，$z$在$Y$上的感受野包括$Y$的全部四个元素，在输入上的感受野包括其中全部$9$个元素。

**可见，我们可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征**。

+ 二维卷积层的核心计算是二维互相关运算。在最简单的形式下，它对二维输入数据和卷积核做互相关运算然后加上偏差。
+ 我们可以设计卷积核来检测图像中的边缘。
+ 我们可以通过数据来学习卷积核。